---
# Tutorial Task: Create a Forward Bank Bill Class #
---
In this tutorial, you will use the Bank Bill and Bond classes to create a Forward Bank Bill class.

### Step 1:

Create some bank bill and bond instruments.

### Step 2:

Create a yield curve using these instruments.

### Step 3:

Design a new Forward Bank Bill class which inherits from the Bank Bill class and uses the yield curve to give an arbitrage-free yield.

---

## Refresher on Classes

### What is a Class
In programming, a class is a way to organize and group data and functions together into a single unit by describing:

- What information the thing should have (its properties)

- What actions the thing should be able to do (its behavior)

The information inside a class is called attributes , and the actions are called methods (which are just functions inside the class).

Once you define a class, you can create multiple objects from it.
Each object will have its own copy of the attributes and will be able to use the methods.

This process of making an object from a class is called instantiation, and the object is called an instance of the class.

### Inheritance
Inheritance is when a new class automatically gets the attributes and methods from an existing class.

The class you start from is called the parent class (or superclass).
The new class you make is called the child class (or subclass).

The child class inherits everything from the parent class and can:

- Use everything the parent already has

- Add new things of its own

- Change (override) things if needed


---

### Step 0:

Import pre-existing classes

In [1]:
import sys
sys.path.append('..')  # Add the parent directory to the system path
from instrument_classes import Bank_bill, Bond, Portfolio
from curve_classes_and_functions import ZeroCurve, YieldCurve

---

### Step 1:

Let's define a few bank bills and bonds.

In [2]:
#we have created bank bills and bonds using classes

In [3]:
bill1 = Bank_bill()
bill1.set_ytm(0.06)
bill1.set_cash_flows()

In [4]:
bill1.get_price()

98.52216748768474

In [5]:
bill2 = Bank_bill(maturity=0.5)
bill2.set_ytm(0.065)
bill2.set_cash_flows()

In [6]:
bond1 = Bond(face_value=100,maturity=1,coupon=0.05,frequency=2)
bond1.set_ytm(0.07)
bond1.set_cash_flows()

In [7]:
bond2 = Bond(face_value=100,maturity=2,coupon=0.06,frequency=1)
bond2.set_ytm(0.075)
bond2.set_cash_flows()

---

### Step 2:

Now we can use the Portfolio class to aggregate the instruments that we have created. Then we pass the portfolio to the YieldCurve class to bootstrap a yield curve.

In [8]:
#Create portfolio class
portfolio = Portfolio()
portfolio.add_bank_bill(bill1)
portfolio.add_bank_bill(bill2)
portfolio.add_bond(bond1)
portfolio.add_bond(bond2)
portfolio.set_cash_flows()

In [9]:
portfolio.get_bonds()

In [10]:
portfolio.get_bank_bills()

In [11]:
#build yield curve based on maturities and associated rate of return
yc = YieldCurve()
yc.set_constituent_portfolio(portfolio)
yc.bootstrap()

PV of all the cashflows except maturity is:  2.4213075060532687
The bond price is:  98.10030572475438
The last cashflow is:  102.5
PV of all the cashflows except maturity is:  5.600721846948358
The bond price is:  97.30665224445646
The last cashflow is:  106.0


In [12]:
yc.get_zero_curve()

([0, 0.25, 0.5, 1, 2],
 [1.0,
  0.9852216748768474,
  0.9685230024213075,
  0.9334536411580596,
  0.8651502867689443])

In [13]:
yc.npv(portfolio)

-5.972513564315916

---

### Step 3:

Now let's try creating our own class. This will be a Forward Bank Bill class, and will have the ability to price a forward bank bill.

In [14]:
class ForwardBankBill(Bank_bill):
    """
    Forward on a bank bill starting in T1 and maturing at T2
    """

    def __init__(self, start, maturity, face_value=100):
        super().__init__(face_value=face_value,ytm=0.0, price=0.0)
        self.start = start
        self.maturity = maturity
        self.term = maturity - start
        self.price = None
        self.ytm = None

    # 1. Determining cash flows
    def set_cash_flows(self):
        self.add_cash_flow(self.start, -self.price)
        self.add_cash_flow(self.maturity, self.face_value)

    # 2. Determine the price and ytm
    def set_fair_yield(self, yield_curve):
        df1 = yield_curve.get_discount_factor(self.start)
        df2 = yield_curve.get_discount_factor(self.maturity)

        self.price = self.face_value * (df2/df1)
        self.ytm = (self.face_value - self.price) / (self.face_value * self.term)

    def get_price(self):
        return self.price
    
    def get_yield(self):
        return self.ytm

In [15]:
fwd = ForwardBankBill(start=0.5, maturity=0.75, face_value=100)
fwd.set_fair_yield(yc)
fwd.set_cash_flows()

In [16]:
fwd.get_price()

np.float64(98.17285187340218)

In [17]:
fwd.get_yield()

np.float64(0.07308592506391279)